In [1]:
pip install --upgrade google-cloud-vision

Note: you may need to restart the kernel to use updated packages.


In [2]:
# Imports the Google Cloud client library
from google.cloud import vision
from io import BytesIO
from tqdm import tqdm
import pandas as pd

In [3]:
def run_quickstart(image_bytes: bytes) -> str:
    """Performs OCR on an image (provided as bytes) using the Cloud Vision API.

    Args:
        image_bytes: The byte content of the image.

    Returns:
        The text detected in the image, or an empty string on error.
    """
    client = vision.ImageAnnotatorClient()

    image = vision.Image(content=image_bytes)

    try:
        response = client.text_detection(image=image)
        text_annotations = response.text_annotations
        if text_annotations:
            return text_annotations[0].description.strip()  # Return the full text, stripped
        else:
            return ""  # Return empty string if no text is found
    except Exception as e:
        print(f"Error during OCR: {e}")
        return ""  # Return empty string on error

In [ ]:
# Replace with your actual values
project_id = 'project_id'
location = "us"
processor_id = "processor_id"
file_path = "example_text_ransaka.jpg"

In [5]:
from datasets import load_dataset

/Users/nevidujayatilleke/Documents/MSC - Research/OCR Comparative Analysis/.venv/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [6]:
dataset = load_dataset("Ransaka/sinhala_synthetic_ocr-large")

In [7]:
print(dataset)

DatasetDict({
    train: Dataset({
        features: ['image', 'text'],
        num_rows: 6969
    })
})


In [8]:
train_dataset = dataset["train"]

In [9]:
train_dataset

Dataset({
    features: ['image', 'text'],
    num_rows: 6969
})

In [10]:
results = []
for i in tqdm(range(len(dataset['train']))):
    item = dataset['train'][i]
    image = item['image']  # PIL Image object
    reference_text = item['text']

    # Convert PIL Image to bytes (PNG format for consistency)
    image_bytes = BytesIO()
    image.save(image_bytes, format="PNG")
    image_content = image_bytes.getvalue()

    generated_text = run_quickstart(image_content)
    results.append({'reference': reference_text, 'generated': generated_text})

 21%|██        | 1435/6969 [28:56<1:35:42,  1.04s/it]

Error during OCR: 503 Getting metadata from plugin failed with error: ('Connection aborted.', ConnectionResetError(54, 'Connection reset by peer'))


 22%|██▏       | 1535/6969 [31:06<2:07:35,  1.41s/it]

Error during OCR: 503 Getting metadata from plugin failed with error: ('Connection aborted.', ConnectionResetError(54, 'Connection reset by peer'))


 47%|████▋     | 3260/6969 [1:08:37<1:05:37,  1.06s/it]

...

 73%|███████▎  | 5118/6969 [1:43:10<40:01,  1.30s/it]

Error during OCR: 503 Getting metadata from plugin failed with error: ('Connection aborted.', ConnectionResetError(54, 'Connection reset by peer'))


 75%|███████▌  | 5242/6969 [1:45:48<46:27,  1.61s/it]  

Error during OCR: 503 Getting metadata from plugin failed with error: ('Connection aborted.', ConnectionResetError(54, 'Connection reset by peer'))


100%|██████████| 6969/6969 [2:17:01<00:00,  1.18s/it]


In [11]:
results

[{'reference': 'ඔහු සතුව තිබී හෙරොයින් කිලෝග්රෑම් 1ක් සොයා ගත් අතර ඔහු',
  'generated': 'ඔහු සතුව තිබී හෙරොයින් කිලෝග්රෑම් 1ක් සොයා ගත් අතර ඔහු'},
 {'reference': 'මේ තරම් මහන්සි වෙලා, ලස්සන අත්දැකීමක් අපිටත් ලගා කරලා දුන්නට දිනේශ් අයියට අනේක වාරයක්',
  'generated': 'තරම් මහන්සි වෙලා, ලස්සන අත්දැකීමක් අපිටත් ලගා කරලා දුන්නට දිනේශ් අයියට අනේක වාරයක්'},
 {'reference': 'සාමාන්යයෙන් කාර් බැටරියකින් මාස දෙකක් පමණ රේඩියෝව අහන්න පුළුවන්. හැබැයි',
  'generated': 'සාමාන්යයෙන් කාර් බැටරියකින් මාස දෙකක් පමණ රේඩියෝව අහන්න පුළුවන්. හැබැයි'},
 {'reference': 'ජනාධිපති මහින්ද රාජපක්ෂට තම වසර 4 ක නිල කාලය සම්පුර්ණ වු විට මැතිවරණයක්',
  'generated': 'ජනාධිපති මහින්ද රාජපක්ෂට තම වසර 4 ක නිල කාලය සම්පුර්ණ වු විට මැතිවරණයක්'},
 {'reference': 'කාර්යාලීය ඇඳුම්, හවස්වරුවේ සාද සඳහා ඇදුම්, ලිනන් හා ඩෙනිම් ඇඳුම්, යට ඇඳුම් පරාසයක් පවතින අතර අවශ්ය',
  'generated': 'దురదు క్రైడ్ ఫెZడ్తాల్, బిలెచలెంGల్ చుడై చలు డ్రాల్, నైట్ లు Gటెల్ డ్రాల్, SO PZCల్ ఆరుబిదద కాలెవెలు రురు లెదాS'},
 {'reference': 'නිවනට යන්න තෘශ්ණාවක් 

In [12]:
df = pd.DataFrame(results)

In [13]:
df.head(20)

,reference,generated
0,ඔහු සතුව තිබී හෙරොයින් කිලෝග්රෑම් 1ක් සොයා ගත්...,ඔහු සතුව තිබී හෙරොයින් කිලෝග්රෑම් 1ක් සොයා ගත්...
1,"මේ තරම් මහන්සි වෙලා, ලස්සන අත්දැකීමක් අපිටත් ල...","තරම් මහන්සි වෙලා, ලස්සන අත්දැකීමක් අපිටත් ලගා ..."
2,සාමාන්යයෙන් කාර් බැටරියකින් මාස දෙකක් පමණ රේඩි...,සාමාන්යයෙන් කාර් බැටරියකින් මාස දෙකක් පමණ රේඩි...
3,ජනාධිපති මහින්ද රාජපක්ෂට තම වසර 4 ක නිල කාලය ස...,ජනාධිපති මහින්ද රාජපක්ෂට තම වසර 4 ක නිල කාලය ස...
4,"කාර්යාලීය ඇඳුම්, හවස්වරුවේ සාද සඳහා ඇදුම්, ලින...","దురదు క్రైడ్ ఫెZడ్తాల్, బిలెచలెంGల్ చుడై చలు డ..."
5,නිවනට යන්න තෘශ්ණාවක් ඕනේ ඒක කුසලමූල තන්හාව.. ඒ...,නිවනට යන්න තෘෂ්ණාවක් ඕනේ ඒක කුසලමූල තන්හාව.. ඒ...
6,", 4 , , , , . 0772104663, 0112845449 කොට්ටාව ක...","0772104663, 0112845449 කොට්ටාව කාමර 4ක, ව.අ. 2..."
7,බුදු සරණයි පුතේ.... ක්ෂුද්‍ර ජීවී විද්‍යාව පිල...,බුදු සරණයි පුතේ.... ක්ෂුද්‍ර ජීවී විද්‍යාව පිල...
8,හරීන්ද්‍ර මම කැමැති චරිතයක්. ඔබට සැප හා ජය. සච...,හරීන්ද්‍ර මම කැමැති චරිතයක්. ඔබට සැප හා ජය. සච...
9,මාධ්ය නිදහස නැ කියලා බෙරිහන් දෙන මහින්ද ප්රමුක...,මාධ්ය නිදහස නැ කියලා බෙරිහන් දෙන මහින්ද ප්රමුක...


In [14]:
df.to_csv("Google_Cloud_Vision_OCR_Results.csv", index = False)

In [15]:
from jiwer import wer, cer

In [16]:
agg_error_wer = 0
agg_error_cer = 0
for i in range(len(df)):
    record = df.iloc[i]
    try:
        error_wer = wer(record['reference'], record['generated'])
        error_cer = cer(record['reference'], record['generated'])
    except:
        print(record)
        print(i)
    agg_error_wer += error_wer
    agg_error_cer += error_cer

reference    None
generated        
Name: 2929, dtype: object
2929


In [17]:
print(f"Word Error Rate (WER): {agg_error_wer/ len(df)}")

Word Error Rate (WER): 0.08866778613398082


In [18]:
print(f"Character Error Rate (CER): {agg_error_cer/ len(df)}")

Character Error Rate (CER): 0.06283137213023841
